In [6]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
# import missingno as msno
# import pingouin as pg
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [7]:
df = sns.load_dataset('titanic')

In [8]:
df.shape

(891, 15)

In [9]:
l_sex = preprocessing.LabelEncoder()
l_sex.fit(["male", "female"])

LabelEncoder()

In [10]:
df.sex = l_sex.transform(df.sex)

In [11]:
y = df.alive
X = df.drop(columns=['alive'])

# NA Replaced with median value in age, MCAR missing

In [12]:
# X.age[X.age.isna()] = X.age.median()
X.age[X.age.isna()] = X.age.mean()

# alternatives
# df.age.fillna(df.age.median(), inplace=True)
# imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
# imp_median.fit(X.age.to_numpy().reshape(-1, 1))
# X.age = imp_median.transform(X.age.to_numpy().reshape(-1, 1))

In [13]:
X.age.isna().sum()

0

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [15]:
regr = LogisticRegression()

# Model training with fit function
regr.fit(X_train[['age', 'sex', 'fare']], y_train)

# Result with the trained model
regr.score(X_test[['age', 'sex', 'fare']], y_test)

0.7713004484304933

In [16]:
print(X.age.mean(), X.age.median())

29.699117647058763 29.69911764705882


# MAR single linear regression

In [17]:
df = sns.load_dataset('titanic')

l_sex = preprocessing.LabelEncoder()
l_sex.fit(["male", "female"])

df.sex = l_sex.transform(df.sex)

y = df.alive
X = df.drop(columns=['alive'])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=4)

In [18]:
X.age.isna().sum()

177

In [19]:
# is there a missing data in fare
X.fare.isna().sum()

0

# Fitting a Linear regression for FARE to age

In [20]:
lreg = LinearRegression()

x = X[X.age.isna() != True]

lreg.fit(x.fare.to_numpy().reshape(-1, 1), x.age)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [21]:
# Nan values that we are going to estimate
age_nan = X[X.age.isna() == True]

In [22]:
pred = lreg.predict(age_nan.fare.to_numpy().reshape(-1, 1))

In [23]:
# assiging predicted values
X.age[X.age.isna() == True] = pred

In [24]:
X.age.isna().sum()

0

In [25]:
# prediction with MAR features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=4)
regr = LogisticRegression()

regr.fit(X_train[['age', 'sex', 'fare']], y_train)
regr.score(X_test[['age', 'sex', 'fare']], y_test)

0.8026905829596412

# Most frequent approach

In [26]:
from sklearn.impute import SimpleImputer

df = sns.load_dataset('titanic')

l_sex = preprocessing.LabelEncoder()
l_sex.fit(["male", "female"])

df.sex = l_sex.transform(df.sex)

y = df.alive
X = df.drop(columns=['alive'])

In [27]:
X

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone
0,0,3,1,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,False
1,1,1,0,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,False
2,1,3,0,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,True
3,1,1,0,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,False
4,0,3,1,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,True
5,0,3,1,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,True
6,0,1,1,54.0,0,0,51.8625,S,First,man,True,E,Southampton,True
7,0,3,1,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,False
8,1,3,0,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,False
9,1,2,0,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,False


In [28]:
imp_mean = SimpleImputer( strategy='most_frequent')

imp_mean.fit(X.age.to_numpy().reshape(-1, 1))
X.age = imp_mean.transform(X.age.to_numpy().reshape(-1, 1))

In [29]:
X

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone
0,0,3,1,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,False
1,1,1,0,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,False
2,1,3,0,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,True
3,1,1,0,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,False
4,0,3,1,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,True
5,0,3,1,24.0,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,True
6,0,1,1,54.0,0,0,51.8625,S,First,man,True,E,Southampton,True
7,0,3,1,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,False
8,1,3,0,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,False
9,1,2,0,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,False


In [30]:
# prediction with MAR features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)
regr = LogisticRegression()

regr.fit(X_train[['age', 'sex', 'fare']], y_train)
regr.score(X_test[['age', 'sex', 'fare']], y_test)

0.7713004484304933

# Calculating Chi^2 distrubution
look at the table made by yusuf bilgin on whatsup 

In [31]:
result = []

In [32]:
x = 177*(342/891)
y = len(df[(df.alive == 'yes') & (df.age.isna() == True)])
print("Expected,", x , 'calculated : ', y )
result.append((pow((x-y), 2))/x)

Expected, 67.93939393939394 calculated :  52


In [33]:
x = 177*(549/891)
y = len(df[(df.alive == 'no') & (df.age.isna() == True)])
print("Expected,", x, 'calculated : ', y)
result.append((pow((x-y), 2))/x)

Expected, 109.06060606060606 calculated :  125


In [34]:
x = 714*(342/891)
y = len(df[(df.alive == 'yes') & (df.age.isna() != True)])
print("Expected,", x, 'calculated : ', y )
result.append((pow((x-y), 2))/x)

Expected, 274.06060606060606 calculated :  290


In [35]:
x = 714*(549/891)
y = len(df[(df.alive == 'no') & (df.age.isna() != True)])
print("Expected,", x, 'calculated : ', y )
result.append((pow((x-y), 2))/x)

Expected, 439.93939393939394 calculated :  424


In [36]:
print(result)
np.sum(result)

[3.739572351525964, 2.329569661606338, 0.927036843445512, 0.5774983614906468]


7.57367721806846

In [37]:
from scipy import stats
p_val = stats.chi2.pdf(7.5736 , df=1)
print(p_val)

0.0032860376848728255


In [67]:
# fig, ax = plt.subplots(1, 1)

# df = 1
# mean, var, skew, kurt = chi2.stats(df, moments='mvsk')

# x = np.linspace(chi2.ppf(0.01, df), chi2.ppf(0.99, df), 100)
# y = chi2.pdf(x, df)
# ax.plot(x, y, 'r-', lw=5, alpha=0.6, label='chi2 pdf')